# 1.0 — Panorama Geral dos dados (Lighthouse) e Pré-Limpeza

**Objetivo:** carregar `lighthouse_imdb.csv`, entender estrutura (tamanho, tipos, nulos) e aplicar **pré-limpeza mínima** em colunas-chave (ano, duração, gross, normalização de título).  

**Saída:** Arquivo limpo e padronizado em `data/processed/lighthouse_clean.csv`

## 1.1 - Imports

In [1]:
# =====================================================
# Imports
# =====================================================

# Manipulação e análise de dados
import math
import numpy as np
import pandas as pd
import re
from pathlib import Path

# Visualização de dados
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from tabulate import tabulate

# Sistema e paths
import os
from pathlib import Path

# Utilidades para notebooks
from IPython.display import display
from IPython.display import Image
from IPython.core.display import HTML

# Manipulação de datas
import datetime

## 1.2 - Funções Auxiliares

In [2]:
class PATHS:
    """
    Caminhos padrão do projeto quando o notebook roda dentro de 'notebooks/'.

    Regra:
    - Se o cwd termina com 'notebooks', a raiz é o pai (../)
    - Caso contrário, assume que já estamos na raiz (útil se abrir o Jupyter na raiz)
    """
    _CWD = Path.cwd()
    ROOT = _CWD.parent if _CWD.name == "notebooks" else _CWD
    RAW  = ROOT / "data" / "raw"
    PROC = ROOT / "data" / "processed"
    INTER = ROOT / "data" / "intermediary"
    REP  = ROOT / "reports"
    FIG  = REP / "figures"

def ensure_dirs():
    """Garante que as pastas principais existam."""
    for p in [PATHS.RAW, PATHS.INTER, PATHS.PROC, PATHS.REP, PATHS.FIG]:
        p.mkdir(parents=True, exist_ok=True)

def set_display(max_cols: int = 100, decimals: int = 2):
    """Ajusta visualização padrão no Pandas/Seaborn para leitura eficiente."""
    pd.options.display.max_columns = max_cols
    fmt = "{:." + str(decimals) + "f}"
    pd.options.display.float_format = fmt.format
    sns.set(style="whitegrid", palette="muted", font_scale=1.1)

In [3]:
# Chamadas de funções
ensure_dirs()
set_display()

## 1.3 - Carregando os Dados

In [4]:
# Realizo a leitura dos dados através da classe
df_lh = pd.read_csv(PATHS.RAW / "lighthouse_imdb.csv", low_memory=False)

## 1.4 - Descrição dos Dados

In [5]:
# Cópia da base - Boa prática para evitar modificações sem intenção
df1 = df_lh.copy()

In [6]:
print("Lighthouse:", df1.shape)

Lighthouse: (999, 16)


In [7]:
print("Lighthouse:", df1.columns)

Lighthouse: Index(['Unnamed: 0', 'Series_Title', 'Released_Year', 'Certificate', 'Runtime',
       'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'Star1',
       'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')


In [8]:
print("\nTipos de dados da base Lighthouse:\n", df1.dtypes)


Tipos de dados da base Lighthouse:
 Unnamed: 0         int64
Series_Title      object
Released_Year     object
Certificate       object
Runtime           object
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes        int64
Gross             object
dtype: object


In [9]:
display(df1.sample(5, random_state=42))

,Unnamed: 0,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
453,454,The Best Years of Our Lives,1946,Approved,170 min,"Drama, Romance, War",8.00,Three World War II veterans return home to sma...,93.00,William Wyler,Myrna Loy,Dana Andrews,Fredric March,Teresa Wright,57259,"23,650,000"
793,794,Hedwig and the Angry Inch,2001,R,95 min,"Comedy, Drama, Music",7.70,A gender-queer punk-rock singer from East Berl...,85.00,John Cameron Mitchell,John Cameron Mitchell,Miriam Shor,Stephen Trask,Theodore Liscinski,31957,"3,029,081"
209,210,Gone Girl,2014,A,149 min,"Drama, Mystery, Thriller",8.10,With his wife's disappearance having become th...,79.00,David Fincher,Ben Affleck,Rosamund Pike,Neil Patrick Harris,Tyler Perry,859695,"167,767,189"
309,310,The Red Shoes,1948,NaN,135 min,"Drama, Music, Romance",8.10,A young ballet dancer is torn between the man ...,NaN,Michael Powell,Emeric Pressburger,Anton Walbrook,Marius Goring,Moira Shearer,30935,"10,900,000"
740,741,Le Petit Prince,2015,PG,108 min,"Animation, Adventure, Drama",7.70,A little girl lives in a very grown-up world w...,70.00,Mark Osborne,Jeff Bridges,Mackenzie Foy,Rachel McAdams,Marion Cotillard,56720,"1,339,152"


In [10]:
df1.nunique(dropna=True)

Unnamed: 0       999
Series_Title     998
Released_Year    100
Certificate       16
Runtime          140
Genre            202
IMDB_Rating       16
Overview         999
Meta_score        63
Director         548
Star1            659
Star2            840
Star3            890
Star4            938
No_of_Votes      998
Gross            822
dtype: int64

In [11]:
display(df1.isna().mean().sort_values(ascending=False)*100)

Gross           16.92
Meta_score      15.72
Certificate     10.11
Unnamed: 0       0.00
Runtime          0.00
Genre            0.00
Series_Title     0.00
Released_Year    0.00
Overview         0.00
IMDB_Rating      0.00
Star1            0.00
Director         0.00
Star2            0.00
Star3            0.00
Star4            0.00
No_of_Votes      0.00
dtype: float64

## 1.5 - Pré-limpeza dos dados da Lighthouse (para matching)

A partir do diagnóstico por coluna, verificou-se que:
1. `Released_Year` aparece como string e eventualmente com sufixos. 
2. `Runtime` vem com sufixo ‘min’. 
3. `Gross` usa vírgula como separador de milhar.

Nesta pré-limpeza, padronizo apenas esses campos e normalizo strings básicas, gerando colunas derivadas, para não atrapalhar na compração do dataset baixado para enriquecer este. Não foi feito imputações ou tratamento de outliers e de valores ausentes — isso fica para a EDA.

In [12]:
df_clean = df1.copy()

In [13]:
# Removo colunas auxiliares de índice geradas ao salvar CSV através de uma lista com um único drop
cols_drop = [c for c in ["Unnamed: 0", "index"] if c in df_clean.columns]
if cols_drop:
    df_clean = df_clean.drop(columns=cols_drop)

In [14]:
# Extraio o ano para facilitar joins por (título, ano) e análises por período
df_clean["Year"] = (
    df_clean["Released_Year"]
      .astype(str) # garante aplicação de operação de strings sem quebrar
      .str.extract(r"(\d{4})", expand=False) # usa regex para capturar a primeira sequênica de 4 digitos
      .astype("Int64") # deixa numérico, o float foi adotado pois aceita NaN caso algum registro não tenha ano
)

In [15]:
# Transformo '142 min' em número para comparar duração entre filmes e usar em modelos
df_clean["Runtime_min"] = (
    df_clean["Runtime"]
      .astype(str)
      .str.extract(r"(\d+)", expand=False) # 
      .astype(float)
)

In [16]:
# Crio Gross_USD (removendo vírgulas e transformando em float)
df_clean["Gross_USD"] = (
    df_clean["Gross"]
      .astype(str)
      .str.replace(",", "", regex=False) # tira o separador de milhar
      .replace({"nan": np.nan}) # se veio a string "nan", troca por NaN real
      .astype(float)
)

In [17]:
# Normalizo títulos para matching e EDA (minúscula, sem pontuação, sem espaços extras)
df_clean["title_norm"] = (
    df_clean["Series_Title"]
      .astype(str)
      .str.lower() # tudo minúsculo
      .str.replace(r"\s+", " ", regex=True) # qualquer sequência de espaços/tab/quebras é substituída por um único espaço
      .str.replace(r"[^\w\s]", "", regex=True) # remove pontuação/sinais (mantém letras/dígitos/_ e espaços)
      .str.strip()
)

In [18]:
# Padroniza categóricas comuns (trim [remover espaços em branco] e NaN coerente)
## Strings limpas evitam problemas nos groupbys, value_counts e joins futuros
for col in ["Certificate", "Genre", "Director", "Star1", "Star2", "Star3", "Star4"]:
    if col in df_clean.columns:
        df_clean[col] = (
            df_clean[col]
              .astype(str)
              .str.strip() # remove espaços sobrando no começo
              .replace({"nan": np.nan}) # garante que o "nan" string vire NaN real
        )


In [19]:
# Removo espaços em variáveis categóricas comuns
for col in ["Certificate", "Genre", "Director", "Star1", "Star2", "Star3", "Star4"]:
    if col in df_clean.columns and df_clean[col].dtype == "O":
        df_clean[col] = df_clean[col].astype(str).str.strip().replace({"nan": np.nan})

In [20]:
# Conferindo a pre-limpeza
print("Tipos (pós pré-limpeza):\n", (df_clean.dtypes))

Tipos (pós pré-limpeza):
 Series_Title      object
Released_Year     object
Certificate       object
Runtime           object
Genre             object
IMDB_Rating      float64
Overview          object
Meta_score       float64
Director          object
Star1             object
Star2             object
Star3             object
Star4             object
No_of_Votes        int64
Gross             object
Year               Int64
Runtime_min      float64
Gross_USD        float64
title_norm        object
dtype: object


In [21]:
print("\n Porcentagem de nulos (pós pré-limpeza):\n", (df_clean.isna().mean().sort_values(ascending=False) * 100))


 Porcentagem de nulos (pós pré-limpeza):
 Gross_USD       16.92
Gross           16.92
Meta_score      15.72
Certificate     10.11
Year             0.10
Released_Year    0.00
Series_Title     0.00
Overview         0.00
Runtime          0.00
IMDB_Rating      0.00
Genre            0.00
Star2            0.00
Star1            0.00
Director         0.00
Star3            0.00
No_of_Votes      0.00
Star4            0.00
Runtime_min      0.00
title_norm       0.00
dtype: float64


In [22]:
display(df_clean.head(3))

,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Year,Runtime_min,Gross_USD,title_norm
0,The Godfather,1972,A,175 min,"Crime, Drama",9.20,An organized crime dynasty's aging patriarch t...,100.00,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",1972,175.00,134966411.00,the godfather
1,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.00,When the menace known as the Joker wreaks havo...,84.00,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",2008,152.00,534858444.00,the dark knight
2,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.00,The early life and career of Vito Corleone in ...,90.00,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",1974,202.00,57300000.00,the godfather part ii


In [23]:
# Testo as duplicatas para match/merge fuituro (por título e ano)
dup_mask = df_clean.duplicated(subset=["title_norm", "Year"], keep=False)
n_dups = dup_mask.sum()
print(f"Duplicatas por (title_norm, Year): {n_dups}")

Duplicatas por (title_norm, Year): 0


## 1.6 - Salvando dataset limpo

In [24]:
# Salvo o dataset limpo e confirmo caminho absoluto
out_path = PATHS.INTER / "lighthouse_clean.csv"
df_clean.to_csv(out_path, index=False)
print(f"Arquivo salvo em: {out_path.resolve()}")

Arquivo salvo em: /home/emersds/repos_projetos/project_lighthouse/data/intermediary/lighthouse_clean.csv


# 1.7 - Decisões e Próximos passos

**Decisões nesta etapa**
- Mantive os nomes originais das colunas para garantir compatibilidade com a base Kaggle.  
- Criei colunas derivadas mínimas (`Year`, `Runtime_min`, `Gross_USD`, `title_norm`) para facilitar análises futuras e possível match entre bases.  
- Nesta etapa não tratei valores ausentes nem outliers; isso fica para a EDA, quando será possível avaliar impacto e estratégia de imputação.  
- Salvei a versão pré-limpa em `data/processed/lighthouse_clean.csv`.

**Próximos passos**
- Carregar e pré-limpar a base Kaggle (`movies_metadata`): criar Year, `title_norm` e `original_title_norm`, converter budget, revenue, runtime, vote_*.  
- Gerar chaves de match: versões deduplicadas por `(title_norm, Year)` e `(original_title_norm, Year)` como fallback.  
- Fazer o matching LH - Kaggle para enriquecer a base e salvar `lighthouse_enriched.csv`.  
- Rodar EDA e métricas descritivas em cima da base enriquecida.  
- Registrar riscos/pendências: `Gross` ausente em parte dos filmes; títulos com variações; anos faltantes em alguns registros.  
